# Load data

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from scipy.spatial import cKDTree

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline as SKPipeline
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors

In [2]:
# Load datasets
train = pd.read_csv("data/Train.csv")
test = pd.read_csv("data/Test.csv")

# ERA5 Climate Conditions at each location
toilets = pd.read_csv("data/toilets.csv") # Information on toilet locations
waste_management = pd.read_csv("data/waste_management.csv") # Additional information on waste management locations in the area.
water_sources = pd.read_csv("data/water_sources.csv") # Additional information on water sources in the area.

In [3]:
# Show unique categories for object columns in train_df
cols = ['Category_Health_Facility_UUID', 'Disease', 'Month', 'Year']

for col in cols:
    print(f"\n{col}:")
    print(f"Number of unique values: {train[col].nunique()}")
    print(f"Unique values: {train[col].unique()}")


Category_Health_Facility_UUID:
Number of unique values: 4
Unique values: ['a9280aca-c872-46f5-ada7-4a7cc31cf6ec'
 'b7f0a600-e19e-4c65-acb3-e28584dae35b'
 'a3761841-2a02-4c17-8589-d35aac4edc24'
 '56cd4cbb-23db-4dde-a6ae-9fc1ed7c8662']

Disease:
Number of unique values: 7
Unique values: ['Dysentery' 'Typhoid' 'Diarrhea' 'Schistosomiasis' 'Malaria'
 'Intestinal Worms' 'Cholera']

Month:
Number of unique values: 12
Unique values: [12 11 10  9  8  7  6  5  4  3  2  1]

Year:
Number of unique values: 4
Unique values: [2022 2021 2020 2019]


In [4]:
# Combine train and test datasets for consistent preprocessing
hospital_data = pd.concat([train, test])

In [5]:
# Drop unnecessary columns from supplementary datasets
for df in [toilets, waste_management, water_sources]:
    df.drop(columns=['Year', 'Month'], inplace=True)

In [6]:
# Rename columns for clarity
def rename_columns(df, prefix):
    for col in df.columns:
        if col not in ['Month_Year_lat_lon', 'lat_lon']:
            df.rename(columns={col: f"{prefix}_{col}"}, inplace=True)

rename_columns(toilets, "toilet")
rename_columns(waste_management, "waste")
rename_columns(water_sources, "water")


In [7]:
# Fill missing values in the 'Total' count of diseases column
hospital_data['Total'].fillna(0, inplace=True)

In [8]:
# Drop rows with missing latitude and longitude in water sources
water_sources.dropna(subset=['water_Transformed_Latitude'], inplace=True)

In [9]:
hospital_data['Date'] = pd.to_datetime(dict(year=hospital_data['Year'], month=hospital_data['Month'], day=1))

# EDA

In [ ]:
# Visualize locations for a specific year and month
# Note the months/year should be in the given timeframe [2019, 2023]
def plot_locations(year=2022, month=1, month_name='January'):
    if year < 2019 or year > 2023:
        print("Invalid year. Please choose a year between 2019 and 2023.")
        return

    if month < 1 or month > 12:
        print("Invalid month. Please choose a month between 1 and 12.")
        return

    if month_name.capitalize() not in ['January', 'February', 'March',
                                       'April', 'May', 'June', 'July',
                                       'August', 'September', 'October',
                                       'November', 'December']:
        print("Invalid month name. Please choose from 'January' to 'December'.")
        return

    plt.figure(figsize=(12, 8))
    subsets = [
        (hospital_data.query(f"Year == {year} and Month == {month}"), 'Transformed', 'Hospital', 's'),
        (water_sources.query(f"water_Month_Year == '{month}_{year}'"), 'water_Transformed', 'Water', 'o'),
        (waste_management.query(f"waste_Month_Year == '{month}_{year}'"), 'waste_Transformed', 'Waste', 'x'),
        (toilets.query(f"toilet_Month_Year == '{month}_{year}'"), 'toilet_Transformed', 'Toilet', '^'),
    ]
    for df, prefix, label, marker in subsets:
        plt.scatter(df[f'{prefix}_Longitude'], df[f'{prefix}_Latitude'], label=label, alpha=0.6, marker=marker)
    plt.title(f'Locations ({month_name.capitalize()} {year})')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_locations()

In [ ]:
# Box plot of Total by Disease
# Sort categories by median Total for clearer ordering
order = hospital_data.groupby('Disease')['Total'].median().sort_values().index.tolist()
data = [hospital_data.loc[hospital_data['Disease'] == c, 'Total'].values for c in order]

plt.figure(figsize=(16, 6))
plt.boxplot(data, tick_labels=order, showfliers=False)
plt.xticks(rotation=90)
plt.xlabel('Disease')
plt.ylabel('Total')
plt.title('Distribution of Total by Disease (sorted by median)')
plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

# Set Date as index
df = hospital_data.set_index("Date")

# Full monthly index over your date range
all_months = pd.date_range(df.index.min(), df.index.max(), freq="MS")  # month start

# Aggregate over locations → regional totals, one column per disease
regional = (
    df
    .groupby("Disease")["Total"]
    .resample("MS")            # monthly start frequency
    .sum()
    .unstack("Disease")       # index = Date, columns = diseases
    .reindex(all_months)      # ensure all months in range
    .fillna(0)                # months with no cases → 0
)

regional.index.name = "Date"

for disease in regional.columns:
    ts = regional[disease]

    # Skip diseases that are always zero
    if ts.sum() == 0:
        continue

    plot_pacf(ts, lags=24)
    plt.title(f"Regional ACF – {disease}")
    plt.show()


In [ ]:
# Count zeros vs non-zeros in hospital_data['Total']
n_total = len(hospital_data)
n_zero = (hospital_data['Total'] == 0).sum()
n_nonzero = (hospital_data['Total'] != 0).sum()

print(f"Total rows: {n_total}")
print(f"Total == 0: {n_zero} ({n_zero / n_total:.2%})")
print(f"Total != 0: {n_nonzero} ({n_nonzero / n_total:.2%})")

In [ ]:
# For each Disease, count Total == 0 and Total != 0 (with percentages)
agg = (
    hospital_data
    .groupby("Disease")["Total"]
    .agg(
        n_total="size",
        n_zero=lambda s: (s == 0).sum(),
        n_nonzero=lambda s: (s != 0).sum(),
    )
    .assign(pct_zero=lambda df: df["n_zero"] / df["n_total"],
            pct_nonzero=lambda df: df["n_nonzero"] / df["n_total"])
    .sort_values("n_zero", ascending=False)
)

# nicer formatting for display
agg[["n_total", "n_zero", "n_nonzero", "pct_zero", "pct_nonzero"]]


In [ ]:
diseases = hospital_data['Disease'].unique()
fig, axes = plt.subplots(len(diseases), 1, figsize=(16, 4 * len(diseases)), sharex=True)

for i, disease in enumerate(diseases):
    ax = axes[i] if len(diseases) > 1 else axes
    df_d = hospital_data[hospital_data['Disease'] == disease]
    locations = df_d['Location'].unique()
    # Aggregate to avoid duplicate entries for pivot
    df_d_agg = df_d.groupby(['Date', 'Location'], as_index=False)['Total'].sum()
    pivot = df_d_agg.pivot(index='Date', columns='Location', values='Total')
    # Plot each location's line
    for loc in locations:
        ax.plot(df_d[df_d['Location'] == loc]['Date'], df_d[df_d['Location'] == loc]['Total'], alpha=0.3, lw=1)
    # IQR shading
    q1 = pivot.quantile(0.25, axis=1)
    q3 = pivot.quantile(0.75, axis=1)
    ax.fill_between(pivot.index, q1, q3, color='grey', alpha=0.3, label='IQR')
    ax.set_title(f"{disease}")
    ax.set_ylabel("Total")
    ax.legend(loc='upper left')
plt.xlabel("Date")
plt.tight_layout()
plt.show()

In [ ]:
# Group by date and Disease and show descriptive stats for Total
total_desc = hospital_data.groupby(['Date', 'Disease'])['Total'].describe()
total_desc = total_desc.sort_index()  # sort by date then disease


display(total_desc)

# Feature Engineering

## Defining a catchment area

A health facility catchment area (simply ‘catchment area’ hereafter), refers to the geographical area served by a health facility1. Knowledge of catchment areas helps to calculate population-based disease incidence and fatality rates2, interrogate the effects of potential risk factors, or assess the effectiveness of interventions employed to disrupt disease transmission. 

In [10]:
# Unique facility-month combos with coordinates
fac_month = (
    hospital_data[["Location", "Year", "Month", "Transformed_Latitude", "Transformed_Longitude"]]
    .drop_duplicates()
    .dropna(subset=["Transformed_Latitude", "Transformed_Longitude"])
)

def add_year_month_from_my(df, col_name, year_col="Year", month_col="Month"):
    m_y = df[col_name].str.split("_", expand=True)
    df[month_col] = m_y[0].astype(int)
    df[year_col]  = m_y[1].astype(int)
    return df

toilets = add_year_month_from_my(toilets, "toilet_Month_Year")
waste_management = add_year_month_from_my(waste_management, "waste_Month_Year")
water_sources = add_year_month_from_my(water_sources, "water_Month_Year")


In [11]:

def build_catchment_env(
    fac_month,
    env_df,
    prefix,
    radius,
    fac_lat_col="Transformed_Latitude",
    fac_lon_col="Transformed_Longitude",
    env_lat_col=None,
    env_lon_col=None,
    year_col="Year",
    month_col="Month",
):

    r_label = int(radius)  # so we get r1, r3, r5, r10

    df_fac = fac_month.copy()
    df_env = env_df.copy()

    if env_lat_col is None:
        env_lat_col = f"{prefix}_Transformed_Latitude"
    if env_lon_col is None:
        env_lon_col = f"{prefix}_Transformed_Longitude"

    # Decide which env columns to aggregate (numeric climate vars)
    exclude_cols = {
        env_lat_col,
        env_lon_col,
        year_col,
        month_col,
        f"{prefix}_Month_Year",
        f"{prefix}_Month_Year_lat_lon",
        "Month_Year",
        "lat_lon",
        "Month_Year_lat_lon",
    }
    value_cols = [
        c for c in df_env.select_dtypes(include=[np.number]).columns
        if c not in exclude_cols
    ]

    # Map original value_col -> "base name" without duplicated prefix
    # e.g. "toilet_tp" -> "tp", so final col is "toilet_tp_catchment_mean_r5"
    base_names = {}
    for c in value_cols:
        if c.startswith(f"{prefix}_"):
            base_names[c] = c[len(prefix) + 1:]  # strip "toilet_" prefix
        else:
            base_names[c] = c

    out_rows = []

    # Loop over each year-month to keep time aligned
    for (yr, mo), fac_sub in df_fac.groupby([year_col, month_col]):
        env_sub = df_env[(df_env[year_col] == yr) & (df_env[month_col] == mo)]

        if env_sub.empty:
            continue

        fac_coords = fac_sub[[fac_lat_col, fac_lon_col]].to_numpy()
        env_coords = env_sub[[env_lat_col, env_lon_col]].to_numpy()

        nn = NearestNeighbors(radius=radius, algorithm="ball_tree")
        nn.fit(env_coords)

        dists_list, idxs_list = nn.radius_neighbors(fac_coords, return_distance=True)

        env_vals = env_sub[value_cols].to_numpy()

        for i, loc_row in fac_sub.reset_index(drop=True).iterrows():
            loc = loc_row["Location"]
            yr_i = loc_row[year_col]
            mo_i = loc_row[month_col]

            idxs = idxs_list[i]
            if len(idxs) == 0:
                row = {
                    "Location": loc,
                    year_col: yr_i,
                    month_col: mo_i,
                    f"n_{prefix}_in_r{r_label}": 0,
                }
                # NaNs for means if no sites in radius
                for c in value_cols:
                    base = base_names[c]
                    row[f"{prefix}_{base}_catchment_mean_r{r_label}"] = np.nan
                out_rows.append(row)
                continue

            vals = env_vals[idxs]         # (n_sites_in_radius, n_features)
            mean_vals = vals.mean(axis=0) # (n_features,)

            row = {
                "Location": loc,
                year_col: yr_i,
                month_col: mo_i,
                f"n_{prefix}_in_r{r_label}": len(idxs),
            }
            for j, c in enumerate(value_cols):
                base = base_names[c]
                row[f"{prefix}_{base}_catchment_mean_r{r_label}"] = mean_vals[j]

            out_rows.append(row)

    catchment_df = pd.DataFrame(out_rows)
    return catchment_df


In [12]:
radius = 1

toilet_catch = build_catchment_env(fac_month, toilets, "toilet", radius)
waste_catch  = build_catchment_env(fac_month, waste_management, "waste", radius)
water_catch  = build_catchment_env(fac_month, water_sources, "water", radius)

merged_data = (
    hospital_data
    .merge(toilet_catch, on=["Location", "Year", "Month"], how="left")
    .merge(waste_catch,  on=["Location", "Year", "Month"], how="left")
    .merge(water_catch,  on=["Location", "Year", "Month"], how="left")
)


A KD Tree is a space-partitioning data structure that organizes points in a k-dimensional space. It is particularly effective for multidimensional search operations such as range searches and nearest-neighbor searches. The tree structure ensures that these operations can be performed efficiently, even on large datasets.

cKDTree (from scipy.spatial) is:
* A specialised, low-level data structure for fast nearest-neighbour lookups in Euclidean space.

Testing three tree structures:
1. 1 nearest neighbour (k) -> irrespective of geographic distance (not fixed)
2. average of 3 nearest neighbour (k) -> irrespective of geographic distance (not fixed)

But the above introduces a conceptual question: if a toilet is 1 km away vs 15 km away, are they equally relevant for the hospital’s disease risk?

3. so we also test out fixed radius

#### Catchment area with cKDTree

In [ ]:
# Finding the single nearest neighbour (non-fixed distance)
def find_nearest(hospital_df, location_df, lat_col, lon_col, id_col):
    # Create a cKDTree for efficient nearest neighbour search
    tree = cKDTree(location_df[[lat_col, lon_col]].values)
    nearest = {}
    # Loop through each hospital and find the nearest site in location_df
    for _, row in hospital_df.iterrows():
        _, idx = tree.query([row['Transformed_Latitude'], row['Transformed_Longitude']])
        nearest[row['ID']] = location_df.iloc[idx][id_col]
    return nearest


In [ ]:
# Ensure unique identifier columns exist in all supplementary datasets
# for df, prefix in [(toilets, 'toilet'), (waste_management, 'waste'), (water_sources, 'water')]:
    # df[f"{prefix}_Month_Year_lat_lon"] = (
        # df[f"{prefix}_Month_Year"] + '_' +
        # df[f"{prefix}_Transformed_Latitude"].astype(str) + '_' +
        # df[f"{prefix}_Transformed_Longitude"].astype(str)
    # )

In [ ]:
# Merge datasets with nearest locations
# merged_data = hospital_data.copy()
# datasets = [
    # (toilets, 'toilet', 'toilet_Month_Year_lat_lon'),
    # (waste_management, 'waste', 'waste_Month_Year_lat_lon'),
    # (water_sources, 'water', 'water_Month_Year_lat_lon'),
# ]

In [ ]:
# for df, prefix, id_col in datasets:
    # nearest = find_nearest(merged_data, df, f"{prefix}_Transformed_Latitude", f"{prefix}_Transformed_Longitude", id_col)
    # nearest_df = pd.DataFrame(list(nearest.items()), columns=['ID', id_col])
    # merged_data = merged_data.merge(nearest_df, on="ID").merge(df, on=id_col)

## Climate anomalies

In [13]:
# --- CLIMATE ANOMALIES BY MONTH (simple climatology) ---

# Ensure Month exists (1–12)
if "Month" not in merged_data.columns:
    merged_data["Month"] = merged_data["Date"].dt.month

# Pick env catchment mean columns (adjust pattern if needed)
env_cols = [c for c in merged_data.columns if "_catchment_mean_r" in c]

for col in env_cols:
    # monthly climatology over all years / locations
    monthly_mean = merged_data.groupby("Month")[col].transform("mean")
    monthly_std  = merged_data.groupby("Month")[col].transform("std")

    merged_data[f"{col}_anom"] = merged_data[col] - monthly_mean
    merged_data[f"{col}_z"] = (
        merged_data[col] - monthly_mean
    ) / (monthly_std + 1e-6)


/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_67516/2586710337.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data[f"{col}_anom"] = merged_data[col] - monthly_mean
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_67516/2586710337.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data[f"{col}_z"] = (
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_67516/2586710337.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

## Outbreak flags

Given that disease outbreak is an anomoly we need to flag with it occurs:
1. is_outbreak_cross_90/95 -> On a given date, this facility’s count is in the top X% of the distribution across all facilities for that disease.
2. is_outbreak_long_3iqe -> For each facility & disease, compare today’s count to that facility’s own historical baseline.

Because diseases have different prevalence patterns, we use disease specific thresholds:
-> For very rare diseases (cholera, dysentery): Any non-zero could be “outbreak”.
-> For common diseases (diarrhoea, intestinal worms): percentile thresholds (top 10–20%) or IQR-based anomalies.

In [14]:
merged_data["Date"] = pd.to_datetime(merged_data["Date"])

merged_data["Total_rank_date_dis"] = (
    merged_data.groupby(["Date", "Disease"])["Total"]
      .rank(method="average", pct=True)
)

cholera_mask = merged_data["Disease"] == "Cholera"
merged_data["is_outbreak_rule"] = 0

mask_chol_valid = cholera_mask & merged_data["Total"].notna()
merged_data.loc[mask_chol_valid, "is_outbreak_rule"] = (
    merged_data.loc[mask_chol_valid, "Total"] > 0
).astype(int)

mask_other_valid = (~cholera_mask) & merged_data["Total_rank_date_dis"].notna()
merged_data.loc[mask_other_valid, "is_outbreak_rule"] = (
    merged_data.loc[mask_other_valid, "Total_rank_date_dis"] >= 0.9
).astype(int)

merged_data = merged_data.sort_values(["Location", "Disease", "Date"])
g = merged_data.groupby(["Location", "Disease"], group_keys=False)

for lag in [1, 2, 3]:
    merged_data[f"is_outbreak_rule_lag{lag}"] = g["is_outbreak_rule"].shift(lag)
    merged_data[f"Total_rank_date_dis_lag{lag}"] = g["Total_rank_date_dis"].shift(lag)


/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_67516/659736596.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data["Total_rank_date_dis"] = (
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_67516/659736596.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data["is_outbreak_rule"] = 0
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_67516/659736596.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

In [15]:
# Long term outbreak

merged_data = merged_data.sort_values(["Location", "Disease", "Date"])
g = merged_data.groupby(["Location", "Disease"], group_keys=False)

roll_med_12 = g["Total"].shift(1).rolling(12, min_periods=6).median()
roll_q25_12 = g["Total"].shift(1).rolling(12, min_periods=6).quantile(0.25)
roll_q75_12 = g["Total"].shift(1).rolling(12, min_periods=6).quantile(0.75)
roll_iqr_12 = roll_q75_12 - roll_q25_12

merged_data["Total_roll_med_12"] = roll_med_12
merged_data["Total_roll_iqr_12"] = roll_iqr_12

k = 3
merged_data["is_outbreak_long_3iqr"] = (
    merged_data["Total"] > (merged_data["Total_roll_med_12"] + k * merged_data["Total_roll_iqr_12"])
).astype(int)


## Time features

In [19]:
def make_time_series_features(
    df: pd.DataFrame,
    id_cols=("Disease", "Location", "Category_Health_Facility_UUID"),
    target_col="Total",
    year_col="Year",
    month_col="Month",
    day_col=None,              # set to a column name if you have Day, else uses 1
    lags=(1, 2, 3, 6, 12),
    rolling_windows=(3, 6),
    add_month_sin_cos=True,
    add_time_index=True,
    drop_na_lags=True,
):

    df = df.copy()

    # --- 1. Build Date column ---
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"])
    else:
        if day_col is not None and day_col in df.columns:
            df["Date"] = pd.to_datetime(
                dict(
                    year=df[year_col].astype(int),
                    month=df[month_col].astype(int),
                    day=df[day_col].astype(int),
                )
            )
        else:
            df["Date"] = pd.to_datetime(
                dict(
                    year=df[year_col].astype(int),
                    month=df[month_col].astype(int),
                    day=1,
                )
            )

    # Ensure consistent sorting
    df = df.sort_values(list(id_cols) + ["Date"])

    # --- 2. Global time_index / diff_date ---
    if add_time_index:
        df = df.sort_values("Date")
        ref_date = df["Date"].min()
        df["time_index"] = (df["Date"] - ref_date).dt.days

    # --- 3. Month + seasonality features ---
    df["month"] = df["Date"].dt.month
    df["year"] = df["Date"].dt.year

    if add_month_sin_cos:
        df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
        df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

    # --- 4. Lag features per series (Disease-Location-Facility) ---
    df = df.sort_values(list(id_cols) + ["Date"])
    grouped = df.groupby(list(id_cols), group_keys=False)

    created_cols = []

    # Lags of the target
    for lag in lags:
        col_name = f"{target_col}_lag{lag}"
        df[col_name] = grouped[target_col].shift(lag)
        created_cols.append(col_name)

    # Rolling stats of the target (using past data only)
    for window in rolling_windows:
        mean_col = f"{target_col}_roll_mean_{window}"
        std_col = f"{target_col}_roll_std_{window}"

        df[mean_col] = grouped[target_col].shift(1).rolling(window).mean()
        df[std_col] = grouped[target_col].shift(1).rolling(window).std()

        created_cols.extend([mean_col, std_col])

        # --- 4b. Same-month-last-year features (if lag 12 is present) ---
    if 12 in lags:
        lag12_col = f"{target_col}_lag12"

        df["Total_same_month_last_year"] = df[lag12_col]
        created_cols.append("Total_same_month_last_year")

    # --- 5. Optionally drop rows with missing lag/rolling features ---
    if drop_na_lags and created_cols:
        df = df.dropna(subset=created_cols).reset_index(drop=True)

    return df

fe_df = make_time_series_features(merged_data)


# Modeling

In [ ]:
# Feature engineering on the FULL merged data (train + 2023 test)
fe_df = make_time_series_features(merged_data)


target_col = "Total"

# Drop junk merge-key columns (but KEEP 'ID' for submission)
drop_obj_cols = [
    "toilet_Month_Year_lat_lon", "toilet_Month_Year",
    "lat_lon_x", "Month_Year_lat_lon_x",
    "waste_Month_Year_lat_lon", "waste_Month_Year",
    "lat_lon_y", "Month_Year_lat_lon_y",
    "water_Month_Year_lat_lon", "water_Month_Year",
    "lat_lon", "Month_Year_lat_lon", "Total_rank_date_dis",
    "is_outbreak_rule", "is_outbreak_cross_90", "is_outbreak_cross_95",
    "is_outbreak_long_3iqr",
]
fe_df = fe_df.drop(columns=[c for c in drop_obj_cols if c in fe_df.columns])

# Sort by time (important for lags + TimeSeriesSplit)
fe_df = fe_df.sort_values("Date").reset_index(drop=True)

# Define train (labelled) vs test (2023 competition) masks
train_mask = (fe_df["Year"] < 2023) & fe_df[target_col].notna()
test_mask  = fe_df["Year"] == 2023

# Define categorical & numeric feature columns
cat_cols = ["Disease", "Location", "Category_Health_Facility_UUID"]
cat_cols = [c for c in cat_cols if c in fe_df.columns]

numeric_cols = [
    c for c in fe_df.select_dtypes(include="number").columns
    if c != target_col
]

feature_cols = numeric_cols + cat_cols

# Build train features / target
X_train = fe_df.loc[train_mask, feature_cols]
y_train = fe_df.loc[train_mask, target_col]

# Build 2023 test features (no y_test here)
X_test = fe_df.loc[test_mask, feature_cols]
ids_test = fe_df.loc[test_mask, "ID"]


# Preprocessor
numeric_transformer = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_transformer = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)

# Define candidate models
models = {
    "RandomForest": RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1,
    ),
    "GradientBoosting": GradientBoostingRegressor(
        random_state=42,
    ),
    "HistGradientBoosting": HistGradientBoostingRegressor(
        random_state=42,
    ),
    "XGBoost": XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        tree_method="hist",
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
    ),
}

tscv = TimeSeriesSplit(n_splits=5)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

results = {}

for name, model in models.items():
    pipe = Pipeline([
        ("prep", preprocess),
        ("model", model),
    ])

    cv_scores = cross_val_score(
        pipe,
        X_train,
        y_train,
        cv=tscv,
        scoring=mae_scorer,
        n_jobs=1,
    )

    mean_mae = -cv_scores.mean()
    std_mae = cv_scores.std()

    results[name] = mean_mae
    print(f"{name} - CV MAE per fold: {-cv_scores}")
    print(f"{name} - Mean CV MAE: {mean_mae:.4f} ± {std_mae:.4f}\n")

print("Summary:")
for name, mae in results.items():
    print(f"{name}: {mae:.4f}")


RandomForest - CV MAE per fold: [8.63415159 7.34646462 6.33592883 7.28234419 5.79654805]
RandomForest - Mean CV MAE: 7.0791 ± 0.9724

GradientBoosting - CV MAE per fold: [8.71543711 7.38791394 6.53826806 7.19527501 5.23411146]
GradientBoosting - Mean CV MAE: 7.0142 ± 1.1368

HistGradientBoosting - CV MAE per fold: [8.17221018 7.28651979 6.28893345 7.04779739 4.83044893]
HistGradientBoosting - Mean CV MAE: 6.7252 ± 1.1219

XGBoost - CV MAE per fold: [8.62305171 7.21284915 6.31752972 7.59709155 4.96716922]
XGBoost - Mean CV MAE: 6.9435 ± 1.2344

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002830 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6181
[LightGBM] [Info] Number of data points in the train set: 3316, number of used features: 402
[LightGBM] [Info] Start training from score 11.745175


/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002421 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9136
[LightGBM] [Info] Number of data points in the train set: 6632, number of used features: 403
[LightGBM] [Info] Start training from score 10.005127


/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003455 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11457
[LightGBM] [Info] Number of data points in the train set: 9948, number of used features: 403
[LightGBM] [Info] Start training from score 9.576498


/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006213 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13588
[LightGBM] [Info] Number of data points in the train set: 13264, number of used features: 405
[LightGBM] [Info] Start training from score 8.895733


/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004486 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15734
[LightGBM] [Info] Number of data points in the train set: 16580, number of used features: 407
[LightGBM] [Info] Start training from score 8.666164
LightGBM - CV MAE per fold: [8.8565534  7.49514605 6.72911634 7.07516431 4.92516398]
LightGBM - Mean CV MAE: 7.0162 ± 1.2708

Summary:
RandomForest: 7.0791
GradientBoosting: 7.0142
HistGradientBoosting: 6.7252
XGBoost: 6.9435
LightGBM: 7.0162


/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import numpy as np

target_col = "Total"

# Sort by time
fe_df = fe_df.sort_values("Date").reset_index(drop=True)

# Masks
train_mask = (fe_df["Year"] < 2023) & fe_df[target_col].notna()
test_mask  = (fe_df["Year"] == 2023)

# Categorical & numeric cols
cat_cols = ["Disease", "Location", "Category_Health_Facility_UUID"]
cat_cols = [c for c in cat_cols if c in fe_df.columns]

# Rebuild col lists from the cleaned fe_df
numeric_cols = [
    c for c in fe_df.select_dtypes(include="number").columns
    if c != target_col
]
cat_cols = ["Disease", "Location", "Category_Health_Facility_UUID"]
cat_cols = [c for c in cat_cols if c in fe_df.columns]
feature_cols = numeric_cols + cat_cols

# Per-disease cat cols (drop Disease itself for per-disease models)
cat_cols_per_dis = [c for c in cat_cols if c != "Disease"]

numeric_transformer = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_transformer = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore")),
])

preprocess_per_dis = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, cat_cols_per_dis),
    ]
)

diseases = fe_df["Disease"].unique()

tscv = TimeSeriesSplit(n_splits=5)
results_by_disease = {}
total_weighted_errors = 0.0
total_samples = 0

for dis in diseases:
    mask_dis = (fe_df["Disease"] == dis) & train_mask
    Xd = fe_df.loc[mask_dis, feature_cols]
    yd = fe_df.loc[mask_dis, target_col]

    n = len(Xd)
    if n < 50:
        print(f"Skipping {dis}: not enough samples ({n}) for robust CV.")
        continue

    print(f"\n=== Disease: {dis} (n={n}) ===")

    pipe = Pipeline([
        ("prep", preprocess_per_dis),
        ("model", HistGradientBoostingRegressor(random_state=42)),
    ])

    maes = []
    for train_idx, val_idx in tscv.split(Xd):
        X_tr, X_val = Xd.iloc[train_idx], Xd.iloc[val_idx]
        y_tr, y_val = yd.iloc[train_idx], yd.iloc[val_idx]

        y_tr_log = np.log1p(y_tr)
        pipe.fit(X_tr, y_tr_log)

        y_pred_log = pipe.predict(X_val)
        y_pred = np.expm1(y_pred_log)

        maes.append(mean_absolute_error(y_val, y_pred))

    mean_mae = float(np.mean(maes))
    results_by_disease[dis] = (mean_mae, n)
    total_weighted_errors += mean_mae * n
    total_samples += n

    print(f"{dis}: Mean CV MAE = {mean_mae:.4f}")

if total_samples > 0:
    weighted_overall_mae = total_weighted_errors / total_samples
    print(f"\nWeighted overall per-disease CV MAE: {weighted_overall_mae:.4f}")

print("\nPer-disease MAEs:")
for dis, (mae, n) in results_by_disease.items():
    print(f"{dis}: {mae:.4f} (n={n})")



=== Disease: Typhoid (n=2170) ===
Typhoid: Mean CV MAE = 7.6117

=== Disease: Diarrhea (n=6500) ===
Diarrhea: Mean CV MAE = 6.9368

=== Disease: Malaria (n=8672) ===
Malaria: Mean CV MAE = 3.4011

=== Disease: Schistosomiasis (n=2169) ===
Schistosomiasis: Mean CV MAE = 2.0591

=== Disease: Intestinal Worms (n=2168) ===
Intestinal Worms: Mean CV MAE = 12.2415

=== Disease: Dysentery (n=2145) ===
Dysentery: Mean CV MAE = 0.4442
Skipping Cholera: not enough samples (24) for robust CV.

Weighted overall per-disease CV MAE: 5.1654

Per-disease MAEs:
Typhoid: 7.6117 (n=2170)
Diarrhea: 6.9368 (n=6500)
Malaria: 3.4011 (n=8672)
Schistosomiasis: 2.0591 (n=2169)
Intestinal Worms: 12.2415 (n=2168)
Dysentery: 0.4442 (n=2145)


In [24]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import (
    HistGradientBoostingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import numpy as np

target_col = "Total"

# Rebuild col lists from the cleaned fe_df, just to be safe
numeric_cols = [
    c for c in fe_df.select_dtypes(include="number").columns
    if c != target_col
]
cat_cols = ["Disease", "Location", "Category_Health_Facility_UUID"]
cat_cols = [c for c in cat_cols if c in fe_df.columns]
feature_cols = numeric_cols + cat_cols

# For per-disease modelling we drop Disease from the categorical block
cat_cols_per_dis = [c for c in cat_cols if c != "Disease"]

numeric_transformer = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_transformer = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore")),
])

preprocess_per_dis = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, cat_cols_per_dis),
    ]
)

# Subset to Intestinal Worms only
dis = "Intestinal Worms"
mask_worms = (fe_df["Disease"] == dis) & train_mask
Xw = fe_df.loc[mask_worms, feature_cols]
yw = fe_df.loc[mask_worms, target_col]

print(f"{dis}: n = {len(Xw)}")

models_worms = {
    "HistGB": HistGradientBoostingRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1,
    ),
    "GradientBoosting": GradientBoostingRegressor(
        random_state=42,
    ),
    "XGBoost": XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        tree_method="hist",
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
    ),
}

tscv = TimeSeriesSplit(n_splits=5)
results_worms = {}

for name, model in models_worms.items():
    print(f"\n=== {dis} – {name} ===")

    pipe = Pipeline([
        ("prep", preprocess_per_dis),
        ("model", model),
    ])

    maes = []

    for train_idx, val_idx in tscv.split(Xw):
        X_tr, X_val = Xw.iloc[train_idx], Xw.iloc[val_idx]
        y_tr, y_val = yw.iloc[train_idx], yw.iloc[val_idx]

        # log-target training
        y_tr_log = np.log1p(y_tr)
        pipe.fit(X_tr, y_tr_log)

        y_pred_log = pipe.predict(X_val)
        y_pred = np.expm1(y_pred_log)

        maes.append(mean_absolute_error(y_val, y_pred))

    mean_mae = float(np.mean(maes))
    results_worms[name] = mean_mae
    print(f"{name} – Mean CV MAE (Intestinal Worms): {mean_mae:.4f}")

print("\nIntestinal Worms – per-model MAE:")
for name, mae in results_worms.items():
    print(f"{name}: {mae:.4f}")


Intestinal Worms: n = 2168

=== Intestinal Worms – HistGB ===
HistGB – Mean CV MAE (Intestinal Worms): 12.2415

=== Intestinal Worms – RandomForest ===
RandomForest – Mean CV MAE (Intestinal Worms): 12.0766

=== Intestinal Worms – GradientBoosting ===
GradientBoosting – Mean CV MAE (Intestinal Worms): 11.7282

=== Intestinal Worms – XGBoost ===
XGBoost – Mean CV MAE (Intestinal Worms): 11.9940

=== Intestinal Worms – LightGBM ===
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000725 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3393
[LightGBM] [Info] Number of data points in the train set: 363, number of used features: 340
[LightGBM] [Info] Start training from score 2.290167
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001372 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11688
[LightGBM] [Info] Number of data points in the train set: 1446, number of used features: 383
[LightGBM] [Info] Start training from score 2.168609


/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001276 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13993
[LightGBM] [Info] Number of data points in the train set: 1807, number of used features: 383
[LightGBM] [Info] Start training from score 2.134019
LightGBM – Mean CV MAE (Intestinal Worms): 13.1894

Intestinal Worms – per-model MAE:
HistGB: 12.2415
RandomForest: 12.0766
GradientBoosting: 11.7282
XGBoost: 11.9940
LightGBM: 13.1894


/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [25]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import (
    HistGradientBoostingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import numpy as np

target_col = "Total"

# Rebuild col lists from the cleaned fe_df (after all drops)
numeric_cols = [
    c for c in fe_df.select_dtypes(include="number").columns
    if c != target_col
]
cat_cols = ["Disease", "Location", "Category_Health_Facility_UUID"]
cat_cols = [c for c in cat_cols if c in fe_df.columns]
feature_cols = numeric_cols + cat_cols

# For per-disease models, we drop Disease from the categorical features
cat_cols_per_dis = [c for c in cat_cols if c != "Disease"]

numeric_transformer = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_transformer = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore")),
])

preprocess_per_dis = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, cat_cols_per_dis),
    ]
)

# Candidate models (same for all diseases)
models_per_dis = {
    "HistGB": HistGradientBoostingRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1,
    ),
    "GradientBoosting": GradientBoostingRegressor(
        random_state=42,
    ),
    "XGBoost": XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        tree_method="hist",
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
    ),
}

diseases = fe_df["Disease"].unique()
tscv = TimeSeriesSplit(n_splits=5)

# results[ disease ][ model_name ] = MAE
results = {}

for dis in diseases:
    # subset to this disease + train period
    mask_dis = (fe_df["Disease"] == dis) & train_mask
    Xd = fe_df.loc[mask_dis, feature_cols]
    yd = fe_df.loc[mask_dis, target_col]

    n = len(Xd)
    if n < 50:
        print(f"\nSkipping {dis}: not enough samples ({n}) for robust CV.")
        continue

    print(f"\n======================")
    print(f"Disease: {dis} (n={n})")
    print(f"======================")

    results[dis] = {}

    for name, model in models_per_dis.items():
        print(f"\n--- {dis} – {name} ---")

        pipe = Pipeline([
            ("prep", preprocess_per_dis),
            ("model", model),
        ])

        maes = []

        for train_idx, val_idx in tscv.split(Xd):
            X_tr, X_val = Xd.iloc[train_idx], Xd.iloc[val_idx]
            y_tr, y_val = yd.iloc[train_idx], yd.iloc[val_idx]

            # log-target training
            y_tr_log = np.log1p(y_tr)
            pipe.fit(X_tr, y_tr_log)

            y_pred_log = pipe.predict(X_val)
            y_pred = np.expm1(y_pred_log)

            maes.append(mean_absolute_error(y_val, y_pred))

        mean_mae = float(np.mean(maes))
        results[dis][name] = mean_mae
        print(f"{dis} – {name}: Mean CV MAE = {mean_mae:.4f}")

print("\n\nSummary per disease & model:")
for dis, model_dict in results.items():
    print(f"\n{dis}:")
    for name, mae in model_dict.items():
        print(f"  {name}: {mae:.4f}")



Disease: Typhoid (n=2170)

--- Typhoid – HistGB ---
Typhoid – HistGB: Mean CV MAE = 7.6117

--- Typhoid – RandomForest ---
Typhoid – RandomForest: Mean CV MAE = 6.7766

--- Typhoid – GradientBoosting ---
Typhoid – GradientBoosting: Mean CV MAE = 7.0180

--- Typhoid – XGBoost ---
Typhoid – XGBoost: Mean CV MAE = 6.9404

--- Typhoid – LightGBM ---
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000837 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3272
[LightGBM] [Info] Number of data points in the train set: 365, number of used features: 339
[LightGBM] [Info] Start training from score 0.727704
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001016 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8870
[LightGBM] [Info] Number of data points in the train set: 1087, number of used features: 382
[LightGBM] [Info] Start training from score 0.763872
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


...

/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001114 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11081
[LightGBM] [Info] Number of data points in the train set: 1431, number of used features: 383
[LightGBM] [Info] Start training from score 0.130138
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001153 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13167
[LightGBM] [Info] Number of data points in the train set: 1788, number of used features: 383
[LightGBM] [Info] Start training from score 0.124143
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Dysentery – LightGBM: Mean CV MAE = 0.4529

Skipping Cholera: not enough samples (24) for robust CV.


Summary per disease & model:

Typhoid:
  HistGB: 7.6117
  RandomForest: 6.7766
  GradientBoosting: 7.0180
  XGBoost: 6.9404
  LightGBM: 7.6623

Diarrhea:
  HistGB: 6.9368
  RandomForest: 6.3018
  GradientBoosting: 6.7009
  XGBoost: 6.6226
  LightGBM: 7.0295

Malaria:
  HistGB: 3.4011
  RandomForest: 3.2111
  GradientBoosting: 3.2917
  XGB

/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [26]:
best_model_name_per_dis = {
    "Typhoid": "RandomForest",
    "Diarrhea": "RandomForest",
    "Malaria": "RandomForest",
    "Schistosomiasis": "RandomForest",
    "Intestinal Worms": "GradientBoosting",
    "Dysentery": "XGBoost",
}

from sklearn.ensemble import (
    HistGradientBoostingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

def make_model(name: str):
    if name == "HistGB":
        return HistGradientBoostingRegressor(random_state=42)
    if name == "RandomForest":
        return RandomForestRegressor(
            n_estimators=300,
            random_state=42,
            n_jobs=-1,
        )
    if name == "GradientBoosting":
        return GradientBoostingRegressor(
            random_state=42,
        )
    if name == "XGBoost":
        return XGBRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            objective="reg:squarederror",
            tree_method="hist",
        )
    if name == "LightGBM":
        return LGBMRegressor(
            n_estimators=500,
            learning_rate=0.05,
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
        )
    raise ValueError(f"Unknown model name: {name}")


In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline as SKPipeline

target_col = "Total"

# after dropping leaky cols and building fe_df
numeric_cols = [
    c for c in fe_df.select_dtypes(include="number").columns
    if c != target_col
]
cat_cols = ["Disease", "Location", "Category_Health_Facility_UUID"]
cat_cols = [c for c in cat_cols if c in fe_df.columns]
feature_cols = numeric_cols + cat_cols

cat_cols_per_dis = [c for c in cat_cols if c != "Disease"]

numeric_transformer = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_transformer = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore")),
])

preprocess_per_dis = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, cat_cols_per_dis),
    ]
)

# masks and ids
train_mask = (fe_df["Year"] < 2023) & fe_df[target_col].notna()
test_mask  = (fe_df["Year"] == 2023)

X_test_full = fe_df.loc[test_mask, feature_cols]
ids_test    = fe_df.loc[test_mask, "ID"]


In [28]:
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import mean_absolute_error  # optional sanity check

diseases = fe_df["Disease"].unique()

# where we’ll store test predictions (for 2023 rows)
y_pred_test = pd.Series(index=fe_df.index[test_mask], dtype=float)

for dis in diseases:
    mask_train_dis = (fe_df["Disease"] == dis) & train_mask
    mask_test_dis  = (fe_df["Disease"] == dis) & test_mask

    Xd_train = fe_df.loc[mask_train_dis, feature_cols]
    yd_train = fe_df.loc[mask_train_dis, target_col]
    Xd_test  = fe_df.loc[mask_test_dis, feature_cols]

    n_train = len(Xd_train)
    n_test  = len(Xd_test)

    print(f"\nDisease: {dis}, train n={n_train}, test n={n_test}")

    if n_test == 0:
        continue  # no test rows for this disease

    # Handle tiny diseases (e.g. Cholera) with a simple rule
    if (dis not in best_model_name_per_dis) or (n_train < 10) or (yd_train.nunique() <= 1):
        print(f"  -> Using fallback 0 prediction for {dis}")
        y_pred_test.loc[Xd_test.index] = 0.0
        continue

    model_name = best_model_name_per_dis[dis]
    model = make_model(model_name)

    print(f"  -> Using model: {model_name}")

    pipe = Pipeline([
        ("prep", preprocess_per_dis),
        ("model", model),
    ])

    # log-target training on ALL pre-2023 data for this disease
    y_tr_log = np.log1p(yd_train)
    pipe.fit(Xd_train, y_tr_log)

    # predict on this disease's 2023 rows
    y_pred_log = pipe.predict(Xd_test)
    y_pred = np.expm1(y_pred_log)
    y_pred_test.loc[Xd_test.index] = y_pred

# Build submission
submission = pd.DataFrame({
    "ID": ids_test,
    "Total": y_pred_test.loc[ids_test.index].values,
})

submission.to_csv("submission_best_per_disease_models.csv", index=False)
submission.head()



Disease: Typhoid, train n=2170, test n=696
  -> Using model: RandomForest

Disease: Diarrhea, train n=6500, test n=696
  -> Using model: RandomForest

Disease: Malaria, train n=8672, test n=696
  -> Using model: RandomForest

Disease: Schistosomiasis, train n=2169, test n=696
  -> Using model: RandomForest

Disease: Intestinal Worms, train n=2168, test n=696
  -> Using model: GradientBoosting

Disease: Dysentery, train n=2145, test n=696
  -> Using model: XGBoost

Disease: Cholera, train n=24, test n=696
  -> Using fallback 0 prediction for Cholera


,ID,Total
23848,ID_41ee755f-c8fb-4605-998d-5dc3d005167c_1_2023...,0.000000
23849,ID_8b36c0ac-b46c-4c9a-af37-d6717faa340e_1_2023...,17.005218
23850,ID_e5b10b72-c677-430e-8eee-289c77eeac0b_1_2023...,0.000300
23851,ID_83880e91-c9db-47f3-8cbb-2a1b485260eb_1_2023...,0.000000
23852,ID_1b04544b-5c96-4053-9a26-a1ab26f8a4a2_1_2023...,0.039568


#### Make predictions on test

In [ ]:
# Fit on all training data (< 2023)
pipe.fit(X_train, y_train)

# Predict for the 2023 competition test set
y_pred_2023 = pipe.predict(X_test)

# Build submission DataFrame
submission = pd.DataFrame({
    "ID": ids_test,          # the ID column from fe_df for 2023 rows
    "Total": y_pred_2023,    # IMPORTANT: column name must be 'Total'
})

submission.to_csv("submission_random_forest.csv", index=False)

print(submission.dtypes)
print(submission.head())